# Overview
For my Mini Project 2 (MP 2), I choose Pathway 1: Exploring Python for ML.

This mini-project, I aim to continue to explore deeper into the Pokemon Datasets (Pokemon index dataset & combat dataset) from my Mini Project 1 by leveraging Machine Learning methods. I used the Testing Hypothesis ML for UX concept and leveraged supervised learning ML methods to predict the likelihood of a pokemon winning a battle. Using the Scikit-learn algorithm cheat-sheet I can find the most applicable regression model to use,implement the models and compare the predicted results with training data and find the Mean Absolute Error of the predictions.

# Methods

Using the Scikit-Learn algorithm cheat sheet, I determine that the best ML methods to use for my dataset are Regression models. In this project, I'll explore Simple Linear Regression and SVR to predict Pokemon Win Percentages. 

1. Simple Linear Regression

Simple linear regression model is used to show or predict the relationship between two variables. The factor that is being predicted is called the dependent variable. The factors that are used to predict the value of the dependent variable are called the independent variables. In this simple model, a straight line approximates the relationship between the dependent variable and the independent variable.

2. SVR Regression 

Support Vector Regression (SVR) is an extension of linear regression that gives us the flexibility to define how much error is acceptable in our model and will find an appropriate line (or hyperplane in higher dimensions) to fit the data.
 

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Analysis: Data Prep
Setting up the data to the stage of manipulation that was completed in mini-project 1


In [ ]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

#import files
#pokemon2 = pd.read_csv("../input/pokemon-index-edited/pokemon2.csv")
pokemon = pd.read_csv("../input/pokemon-challenge/pokemon.csv")
combat = pd.read_csv("../input/pokemon-challenge/combats.csv")
#tests = pd.read_csv("../input/pokemon-challenge/tests.csv")
# rename the column in pokemon data with "#" as "number" as its name
pokemon = pokemon.rename(index=str, columns={"#": "Number"})
# Find total win number
total_Wins = len(combat.Winner.value_counts())
# get the number of wins for each pokemon
numberOfWins = combat.groupby('Winner').count()
countByFirst = combat.groupby('Second_pokemon').count()
countBySecond = combat.groupby('First_pokemon').count()
# Finding the total fights of each pokemon
numberOfWins['Total Fights'] = countByFirst.Winner + countBySecond.Winner
# Finding the win percentage of each pokemon
numberOfWins['Win Percentage']= numberOfWins.First_pokemon/numberOfWins['Total Fights']
print(numberOfWins)
# Merge the the original pokemon dataset with the winning dataset
results2 = pd.merge(pokemon, numberOfWins, right_index = True, left_on='Number')
results3 = pd.merge(pokemon, numberOfWins, left_on='Number', right_index = True, how='left')




In [ ]:
#plot graph of Speed vs Win Percentage
import matplotlib.pyplot as plt
sns.regplot(x="Speed", y="Win Percentage", data=results3, logistic=True).set_title("Speed vs Win Percentage")

# Analysis: Simple Linear Regression
Based on what I found in previous my previous project, I know that the Speed of the Pokemon strongly correlations to the Win Percentage. Using Linear Regression and existing Speed & Win Percentage data, I can predict the Win Percentage of Pokemons who currently has no Win Percentage using their Speed. 

In [ ]:
#import libraries to facilitate code testing
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline
import pandas as pd
import numpy as np


In [ ]:
#Find the pokemons without Win Percentage data 
results3[results3['Win Percentage'].isnull()]

In [ ]:
#Implement Linear Regression
from sklearn.linear_model import LinearRegression
results4=results3.dropna()

x = results4["Speed"].values.reshape(-1,1)
y = results4['Win Percentage'].values.reshape(-1,1)

plt.scatter(x,y, s= 10)
plt.title("Speed v.s Win Percentage ")
plt.xlabel("Speed")
plt.ylabel("Win Percentage")

model=LinearRegression()
model.fit(x,y)
y_pred = model.predict(x)
plt.scatter(x,y, s= 10)
plt.plot(x,y_pred,color="purple")

In [ ]:
# Predict the Win Percentage of Blastoise (Speed=78)
y = 78 * model.coef_ + model.intercept_
y

In [ ]:
#let's check the win percentage with other pokemons with the same Speed

R1=results3[results3['Speed']==78]

R1

Conclusion: 
Our Linear Regression Model predicts that the Win Percentage of Blastoise (Speed=78) is 0.59. 

# Analysis: SVR
Now that I've tried Simple Linear Regression, I want to try a more indepth ML approach and start with splitting the datasets into Training Set and Testing Set. According to the Scikit-Learn algorithm cheat sheet, I can use SVR for my dataset. 

In [ ]:
# Splitting the dataset into the Training set and Test set (80:20 split)
X = results4.iloc[:, 5:11].values
Y = results4.iloc[:, 15].values
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
    
# Fitting SVR to the dataset
from sklearn.svm import SVR
regressor = SVR(kernel = 'linear')
regressor.fit(X_train, Y_train)
print(regressor.score(X_train, Y_train))


Next, we will predict output and compare actual Win Percentages with predicted Win Percentages

In [ ]:
#Predict Output and compare Actual Win Percentages with predicted Win Percentages
Y_pred= regressor.predict(X_test)
df = pd.DataFrame({'Actual': Y_test.flatten(), 'Predicted': Y_pred.flatten()})
df

To get a better sense of the comparision result, we can visualize the comparison result as a bar graph. Since there are too many rows, we will only visualize the first 25 rows. 

In [ ]:
#Draw bar graph of Actual vs Predicted results
df1 = df.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
#get the Mean Absolute Error of the predictions
from sklearn.metrics import mean_absolute_error
from math import sqrt
mae = mean_absolute_error(Y_test, Y_pred)
print("Mean Absolute Error: " + str(mae))

# Conclusion

I had a fun time exploring the two regression methods. The Simple Linear Regression method is great when I know the one factor that strongly correlates to the variable I am predicting. With the SVR method, I approached it from a more proper ML way by splitting the dataset into Train Set and Test Set, trained the model, used the model to predict Win Percentages, and then compared the predicted Win Percentages with actual Win Percentages. To determine the accuracy of the results I also calculated the mean absolute error of the results. 

To futher the exploration, I would try more regression methods and compare the mean absolute error results with that of the SVR's to determine if other methods can be more accurate. 